# <div align="center">CP322-A Mini-Project 1: Machine Learning</div>
## <div align="center">Group 6</div>
### <div align="center">due on 15-Oct-2023 at 11:30 PM</div>

Imports:

In [ ]:
import numpy as np

## Task 1: Acquire, preprocess, and analyze the data

1. Load the datasets into NumPy objects (i.e., arrays or matrices) in Python. Remember to convert the wine dataset
to a binary task, as discussed above.
2. Clean the data. Are there any missing or malformed features? Are there other data oddities that need to be
dealt with? You should remove any examples with missing or malformed features and note this in your
report. For categorical variables, you can use a one-hot encoding.
3. Compute basic statistics on the data to understand it better. E.g., what are the distributions of the positive vs.
negative classes, what are the distributions of some of the numerical features? what are the correlations between
the features? how do the scatter plots of pair-wise features look like for some subset of features?

In [ ]:
def readFile(filename):
    data = []
    labels = []
    with open(filename, "r") as file:
        for line in file:
            line = line.strip()
            if line:  # Skip empty lines
                row = line.split(",")
                if filename == "data/adult.data":
                    # Convert non-numerical features to float
                    age = float(row[0])
                    fnlwgt = float(row[2])
                    education_num = float(row[4])
                    capital_gain = float(row[10])
                    capital_loss = float(row[11])
                    hours_per_week = float(row[12])
                    # Combine the numerical features
                    numerical_features = [age, fnlwgt, education_num, capital_gain, capital_loss, hours_per_week]
                    # Append the numerical features
                    data.append(numerical_features)
                    label = row[-1]
                    # Map the labels to binary values, e.g., '<=50K' to 0 and '>50K' to 1
                    labels.append(0 if label == ' <=50K' else 1)
                elif filename == "data/iris.data":
                    data.append([float(val) for val in row[:-1]])
                    label = row[-1]
                    if label == 'Iris-setosa':
                        labels.append(0)
                    elif label == 'Iris-versicolor':
                        labels.append(1)
                    elif label == 'Iris-virginica':
                        labels.append(2)
                else:
                    data.append(row[:-1])
                    label = row[-1]
                    labels.append(0 if label == 'b' else 1)

    data = np.array(data)

    return data, labels


### Dataset 1 (Ionosphere): 

In [ ]:
filename = "data/ionosphere.data"

data,labels = readFile(filename)

# Count the number of positive class instances
positive_count = sum(1 for label in labels if label == 1)

# Count the number of negative class instances
negative_count = sum(1 for label in labels if label == 0)

#what are the distributions of the positive vs. negative classes?
print("Distribution of classes:")
print("Positive (g):", positive_count)
print("Negative (b):", negative_count)

print("\nData:")
print(data,labels)

#what are the distributions of some of the numerical features?

### Dataset 2 (Adult Data Set):

In [ ]:
filename = "data/adult.data"

data, labels = readFile(filename)

# Count the number of positive class instances
positive_count = sum(1 for label in labels if label == 1)

# Count the number of negative class instances
negative_count = sum(1 for label in labels if label == 0)

print("Distribution of classes:")
print("Positive (>50):", positive_count)
print("Negative (<=50):", negative_count)

print("\nData:")
print(data, labels)

#what are the distributions of some of the numerical features?

### Dataset 3 (Iris Data Set):

In [ ]:
filename = "data/iris.data"
iris_data, iris_labels = readFile(filename)

# X = np.array(iris_data)
# Y = np.array(labels)

print(iris_data)
print(iris_labels)

### Dataset 4 (Choice):

## Task 2: Implement the models

#### 1. Implement logistic regression, and use (full batch) gradient descent for optimization.
#### 2. Implement k-Nearest Neighbor (KNN), and find the best K.

Implement both models as Python classes. You should use the constructor for the class to initialize the model
parameters as attributes, as well as to define other important properties of the model.
• Each of your models’ classes should have (at least) two functions:
– Define a fit function, which takes the training data (i.e., x and y)—as well as other hyperparameters (e.g.,
the learning rate and/or number of gradient descent iterations)—as input. This function should train your
model by modifying the model parameters.
– Define a predict function, which takes a set of input points (i.e., x) as input and outputs predictions (i.e.,
yˆ) for these points. Note that for linear regression you need to convert probabilities to binary 0-1
predictions by thresholding the output at 0.5!
In addition to the model classes, you should also define functions evaluate_acc to evaluate the model accuracy.
This function should take the true labels (i.e., y), and target labels (i.e., yˆ) as input, and it should output the accuracy
score.
• Lastly, you should implement a script to run k-fold cross-validation

### Logistic Regression:
Melissa
Grant
Yvonne

In [ ]:
class LogisticRegression:
    def __init__(self):
        self.learning_rate = 0.01
        self.num_iterations = 1000
        self.weights = None
        self.bias = None

    def sigmoid(self, z):
        # Sigmoid function to convert values to probabilities between 0 and 1
        return 1 / (1 + np.exp(-z)) #sigmoid(z) = 1 / ( 1 + e( - z ) )

    def fit(self, data, labels): #training the logistic regression model
        num_samples, num_features = data.shape
        self.weights = np.zeros(num_features)
        self.bias = 0
        
        converge=0.0001
        converged = False
        cost1 = 1
       
        while not converged:
        # Gradient descent
        #for i in range(self.num_iterations):
            #Hypothesis Function
            linear_model = np.dot(data, self.weights) + self.bias
            predictions = self.sigmoid(linear_model)

            # Compute gradients
            #∂J/∂w = (1/m) * Σ[(h(x) - y) * x] , ∂J/∂b = (1/m) * Σ(h(x) - y)

            dw = (1/num_samples) * np.dot((predictions - labels),data)
            db = (1/num_samples) * np.sum(predictions - labels)

            # Update the parameters in the opposite direction of the gradient
            #w := w - α * ∂J/∂w  ,  b := b - α * ∂J/∂b
            self.weights -= self.learning_rate * dw
            self.bias -= self.learning_rate * db
            cost = 0
        
            for i in range (num_samples):
                #cost += -(labels*np.log(predictions)+(1-labels)*np.log(1-predictions))
                cost+= -1/num_samples * np.sum(np.dot(labels[i],np.log(predictions[i])+ np.dot((1-labels[i]),np.log(1-predictions[i])))) 
                
            #print("cost",cost1-cost)   
            if abs(cost1-cost)<=converge:
                converged = True
            cost1=cost
            
            return
        
        
            
    def predict(self, data):
        #Hypothesis Function
        linear_model = np.dot(data, self.weights) + self.bias
        predictions = self.sigmoid(linear_model)
        return [1 if p >= 0.5 else 0 for p in predictions]

    def evaluate_acc(self, label_true, label_pred):
        correct = np.sum(label_true == label_pred)
        total = len(label_true)
        return correct / total

filename = "adult.data"

data, labels = readFile(filename)

# Combine features and labels
data_with_labels = list(zip(data, labels))

# Split data into training and testing sets (80% training, 20% testing)
split_ratio = 0.7
split_index = int(len(data_with_labels) * split_ratio)

train_data, train_labels = zip(*data_with_labels[:split_index])
test_data, test_labels = zip(*data_with_labels[split_index:])

model = LogisticRegression()

# Fit the model to the training data
model.fit(np.array(train_data), np.array(train_labels))

# Make predictions on the test data
labels_pred = model.predict(np.array(test_data))

# Evaluate the model's accuracy
accuracy = model.evaluate_acc(np.array(test_labels), labels_pred)
print(f"Accuracy: {accuracy:.2f}")



def k_fold (data, labels, k):
    
    accuracies = []
    index_length = len(data)//k
    counter = 0
    model = LogisticRegression()
    
    for i in range(k):
        if i == k-1:
           
            data_testing_set = data[counter:]
            data_training_set = data[:counter]
            label_testing_set = labels[counter:]
            label_training_set = labels[:counter]
            
        else:
            data_testing_set = data[counter:index_length+counter]
            label_testing_set = labels[counter:index_length+counter]
            if counter == 0:
                data_training_set = data[index_length+1:]
                label_training_set = labels[index_length+1:]
            else:
                data_training_set = np.concatenate((data[0:counter] , data[index_length+counter+1:]))
                label_training_set = np.concatenate((labels[0:counter] , labels[index_length+counter+1:]))
        counter+=index_length

        model.fit(np.array(data_training_set),np.array(label_training_set))
        labels_pred = model.predict(np.array(data_testing_set))
        accuracy = model.evaluate_acc(np.array(label_testing_set),np.array(labels_pred))
        accuracies.append(accuracy)
    
    return accuracies


kfold = k_fold(data,labels, k = 4)
print("kfold",kfold)

### K-Nearest Neighbor (KNN):

## Task 3: Run Experiments

The goal of this project is to have you explore linear classification and compare different features and models. Use
5-fold cross-validation to estimate performance in all of the experiments. Evaluate the performance using accuracy.
You are welcome to perform any experiments and analyses you see fit (e.g., to compare different features), but at a
minimum, you must complete the following experiments in the order stated below:

#### 1. Compare the accuracy of k-nearest neighbor and logistic regression on the four datasets.

#### 2. Test different k values for the k-nearest neighbor to find the best k-value by showing the accuracy plot. 

#### 3. Test different learning rates for gradient descent applied to logistic regression. Use a threshold for change in the value of the cost function as termination criteria and plot the accuracy on the train/validation set as a function of iterations of gradient descent.

#### 4. Compare the accuracy of the two models as a function of the size of the dataset (by controlling the training size)

Note: The above experiments are the minimum requirements that you must complete; however, this project is open-ended. For example, you might investigate different stopping criteria for gradient descent in logistic regression and develop an automated approach to select a good subset of features. You do not need to do all of these things, but you should demonstrate creativity, rigor, and an understanding of the course material in how you run your chosen experiments and how you report on them in your write-up.